# Creating the First Commitment Transaction

In this section we'll create the First Commitment transaction of a Lightning Channel from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A high level understanding of the Lightning Network. e.g. [Mastering Lightning](https://github.com/bitcoinbook/bitcoinbook) TODO Get the chapter to point it here
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).

- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'

## Commitment Transactions

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md))  details the exact format of on-chain transactions, which both sides need to agree on to ensure signatures are valid. This consists of the funding transaction output script, the commitment transactions, and the HTLC transactions.

This notebook will focus on commitment transactions.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'

Alice send the `open_channel` message to Bob. This message define many channel operational criterias, here we are going to focus on the information that are used in the commitment transaction:
* push_msat
* dust_limit_satoshis
* feerate_per_kw
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

If Bob does not agree with the criterias sent into the `open_channel` message he send a [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice, otherwise he sends the [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, containing some operational criterias from your side, here we are going to focus on the information that are used in the commitment transaction:
* dust_limit_satoshis
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

## Setup
### Requirements

For this exercise we'll run the previous notebook to create the channel funding transaction beetween Alice and Bob.

Below, run the notebook:

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 1 - Funding Transactions/Funding-Transaction.ipynb"

Stored 'alice_funding_address' (str)
Alice's pubkey: 037685b7e0a23a1b58540e6af6f60550e2ed6705e858817d92df23170a65e4a4c1
Alice's funding p2wpkh address: bcrt1qk7k7fzan0wfn6tf3y4vylujlvcrs2s5v4d2ngm
Alice's change p2wpkh address: bcrt1qhm34rrrhcntahx9cg8dzsgxeq9xymnkh8rgcrk
2025-01-08T00:47:06.303000Z TestFramework (INFO): PRNG seed is: 8570498221784578339
2025-01-08T00:47:06.305000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_7ut6h3i2
txid_to_spent: 6aee28cb2f722b1c1064b04c08d0de897bc4c83ef240351f5e686a2bc00e50f5, 1
Bob's pubkey: 03e81dfa1a26011178a4c5e8a704e5909015a51558deb011c43d98fd94c63b1223
Alice pubkey is lexicographically lesser than Bob pubkey:
037685b7e0a23a1b58540e6af6f60550e2ed6705e858817d92df23170a65e4a4c1 < 03e81dfa1a26011178a4c5e8a704e5909015a51558deb011c43d98fd94c63b1223
Redeem Script:
5221037685b7e0a23a1b58540e6af6f60550e2ed6705e858817d92df23170a65e4a4c12103e81dfa1a26011178a4c5e8a704e5909015a51558deb011c43d98fd94c63b122352ae
channel_address: 

### Creating the Commitment Transactions

Note that the funding transaction was not published yet, as without the commitment transaction signed by Bob, Alice would have to trust Bob to get back her funds from the multisig UTXO.

Now we can start creating Alice unsigned commitment transaction. What makes a transaction 'unsigned' is that the witness field is empty.

#### The Commitment Number
To more easily allow both sides to keep track of the commitment numbers, each commitment actually encodes the number of the commitment within the lock time and sequence fields of the commitment transaction. Within the protocol, this special encoding is referred to as **state hints**. Assuming a party knows the current commitment number, they’re able to use the state hints to easily recognize if a broadcasted commitment was a revoked one. Rather than encode the state hint in plain sight, an obfuscated state hint is used in its place. This obfuscation is achieved by first XORing the current commitment number with a set of random bytes generated deterministically using the funding public keys of both sides of the channel. A total of 6 bytes (48 bits) across the lock time and sequence (24 bits of the locktime and 24 bits of the sequence) are used to encode the state hint within the commitment transaction, so 6 random bytes are needed to use for XORing. To obtain these 6 bytes, both sides obtain the SHA-256 hash of the initiator’s funding key concatenated to the responder’s funding key.

In [2]:
# The 48-bit commitment number is obscured by XOR with the lower 48 bits of
# SHA256(payment_basepoint from open_channel || payment_basepoint from accept_channel)

# Commitment number on the opening channel 
commitment_number = 0 

to_obscure = sha256(alice_funding_pubkey + bob_funding_pubkey)
to_obscure_int = int.from_bytes(to_obscure, byteorder='big')
# Extract lower 48 bits
to_obscure_int_lower48 = to_obscure_int & 0xFFFFFFFFFFFF
# obscured commitment number is result of xor operation 
commitment_number_obscured = to_obscure_int_lower48 ^ commitment_number

In [3]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Calculate the txid of the funding channel tx created on the previous notebook
funding_channel_txid = hash256(unsigned_tx)
# The index of the funding channel tx createt on the previous notebook
funding_channel_txindex = 1
# Convert txid and index to bytes (little endian)
funding_channel_txid = (funding_channel_txid)[::-1]
funding_channel_txindex = funding_channel_txindex.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are upper 24 bits of the obscured commitment number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (1 byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

### Transaction Outputs

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

As this is the first commitment transaction it will have no outputs to Bob, because Alice was the funder of the channel and she is not sending any Sats to Bob, if she wanted to do that she could use the field `push_msat` on `open_channel` message and define the value in mili satoshis. As the channel was not openned yet, there is no offered or received HTLC, so none HTLC outputs will be created also. So the first Commitment Transaction outputs for Alice and Bob will be the following.

Alice first Commitment Transaction will have two outputs:
* to_local_anchor_output
* to_local_output

Bob first Commitment Transaction will have two outputs:
* to_remote_anchor_output
* to_remote_output

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md))  defines that outputs in transactions are always sorted first according to their value, this way the to_local_anchor_outuput is the first one, as its amount output is fixed at 330 sats, the default dust limit for P2WSH.

**anchor outputs** exists to prevent a malicious peer from attaching child transactions with a low fee density to an anchor and thereby blocking the victim from getting the commit tx confirmed in time. This defense is supported by a change in Bitcoin core 0.19: [bitcoin/bitcoin#15681](https://github.com/bitcoin/bitcoin/pull/15681). This is also the reason that every non-anchor output on the commit tx is CSV locked. The feature is optional and can be enabled if both peers in a channel support it. But [Eclair v0.11.0](https://github.com/ACINQ/eclair/blob/master/docs/release-notes/eclair-v0.11.0.md) stop accepting channels that don't support anchor outputs.

### Key Derivations

Each commitment transaction uses a unique localpubkey, local_delayedpubkey and revocationpubkey. Those are changed for every transaction based on the per_commitment_point.


The **localdelayed** pubkeys are simply generated by addition from their base points. As definet at Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G
```

The **revocationpubkey** is a blinded key, Alice uses its own `revocation_basepoint` and the Bob node's `first_commitment_point`, sent into `accept_channel` message, to derive the revocationpubkey for the commitment. As definet at Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#revocationpubkey-derivation)):

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```



In [5]:
# ALICE DELAYED PUBKEY
# alice_delayed_pubkey = alice_delayed_payment_basepoint + SHA256(per_commitment_point || alice_delayed_payment_basepoint) * G
# alice_delayed_payment_basepoint is generate by alice and sent into the `open_channel` message
alice_delayed_payment_basepoint_compressed = "036d6caac248af96f6afa7f904f550253a0f3ef3f5aa2fe6838a95b216691468e2"
alice_delayed_payment_basepoint = decompress_point(alice_delayed_payment_basepoint_compressed)
# per_commitment_point is generated by alice and sent into the `open_channel` message
alice_per_commitment_point_compressed = "025f7117a78150fe2ef97db7cfc83bd57b2e2c0d0dd25eaf467a4a1c2a45ce1486"
alice_per_commitment_point = decompress_point(alice_per_commitment_point_compressed)
# Compute SHA256(per_commitment_point || basepoint)
# Concatenate the x-coordinates of per_commitment_point and basepoint
sha_output = sha256(bytes.fromhex(alice_per_commitment_point_compressed) + bytes.fromhex(alice_delayed_payment_basepoint_compressed))
sha_int = int.from_bytes(sha_output, 'big') % n

alice_delayed_pubkey_point = alice_delayed_payment_basepoint +  sha_int * G
alice_delayed_pubkey = compress_pubkey(alice_delayed_pubkey_point)
print(alice_delayed_pubkey)

# BOB REVOGATION PUBKEY
# revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
# bob_revocation_basepoint is generate by bob and sent into the `accept_channel` message
bob_revocation_basepoint_compressed = "036d6caac248af96f6afa7f904f550253a0f3ef3f5aa2fe6838a95b216691468e2"
bob_revocation_basepoint = decompress_point(bob_revocation_basepoint_compressed)
# per_commitment_point is generated by bob and sent into the `accept_channel` message
bob_per_commitment_point_compressed = "025f7117a78150fe2ef97db7cfc83bd57b2e2c0d0dd25eaf467a4a1c2a45ce1486"
bob_per_commitment_point = decompress_point(bob_per_commitment_point_compressed)
# Compute SHA256(revocation_basepoint || per_commitment_point)
sha1_output = sha256(bytes.fromhex(bob_revocation_basepoint_compressed) + bytes.fromhex(bob_per_commitment_point_compressed))
print(sha1_output.hex())
sha1_int = int.from_bytes(sha1_output, 'big') % n
# now multiply it by revocation_basepoint
part1 = bob_revocation_basepoint * sha1_int
print(compress_pubkey(part1))
# Compute SHA256(per_commitment_point || revocation_basepoint)
sha2_output = sha256(bytes.fromhex(bob_per_commitment_point_compressed) + bytes.fromhex(bob_revocation_basepoint_compressed))
print(sha2_output.hex())
sha2_int = int.from_bytes(sha2_output, 'big') % n
# now multiply it by per_commitment_point
part2 = bob_per_commitment_point * sha2_int
print(compress_pubkey(part2))
bob_revocation_pubkey_point = part1 + part2
bob_revocation_pubkey = compress_pubkey(bob_revocation_pubkey_point)
print(bob_revocation_pubkey)


0235f2dbfaa89b57ec7b055afe29849ef7ddfeb1cefdb9ebdc43f5494984db29e5
efbf7ba5a074276701798376950a64a90f698997cce0dff4d24a6d2785d20963
02c00c4aadc536290422a807250824a8d87f19d18da9d610d45621df22510db8ce
cbcdd70fcfad15ea8e9e5c5a12365cf00912504f08ce01593689dd426bca9ff0
0325ee7d3323ce52c4b33d4e0a73ab637711057dd8866e3b51202a04112f054c43
02916e326636d19c33f13e8c0c3a03dd157f332f3e99c317c141dd865eb01f8ff0


In [6]:
# OUTPUTS FOR ALICE
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# ANCHOR AMOUNT OUTPUT
anchor_output_value = 330

# ANCHOR OUTPUT  
alice_anchor_output_value = anchor_output_value.to_bytes(8, byteorder="little", signed=True)
# ANCHOR P2WSH
# <local_funding_pubkey/remote_funding_pubkey> OP_CHECKSIG OP_IFDUP
# OP_NOTIF
#     OP_16 OP_CHECKSEQUENCEVERIFY
# OP_ENDIF
to_alice_anchor_redeemScript = bytes.fromhex(
    "21"
    + alice_funding_pubkey.hex()
    + "ac"  # OP_CHECKSIG
    + "73"  # OP_IFDUP
    + "64"  # OP_NOTIF
    + "60"  # OP_16
    + "B2"  # OP_CHECKSEQUENCEVERIFY
    + "68") # OP_ENDIF

to_alice_anchor_script_hash = sha256(to_alice_anchor_redeemScript)
to_alice_anchor_output_spk = bytes.fromhex("0020") + to_alice_anchor_script_hash

# TO_LOCAL OUTPUT
# This output sends funds back to the owner of this commitment transaction and thus must be timelocked
# using OP_CHECKSEQUENCEVERIFY. It can be claimed, without delay, by the other party if they know the
# revocation private key. The output is a version-0 P2WSH, with the witness script below:
# OP_IF
    # Penalty transaction
#    <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG
to_alice_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + bob_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + alice_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

to_alice_script_hash = sha256(to_alice_redeemScript)
to_alice_output_spk = bytes.fromhex("0020") + to_alice_anchor_script_hash



### Fee Calculation

The fee calculation for commitment transactions  is based on the current `feerate_per_kw` sent by Alice into the `open_channel` message and the **expected** weight of the transaction.

The Basis of Lightning Technology [BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#fees) defines the following for the expected weights:
* Commitment weight (no option_anchors):   724 + 172 * num-untrimmed-htlc-outputs
* Commitment weight (option_anchors):     1124 + 172 * num-untrimmed-htlc-outputs
* HTLC-timeout weight (no option_anchors): 663
* HTLC-timeout weight (option_anchors): 666
* HTLC-success weight (no option_anchors): 703
* HTLC-success weight (option_anchors): 706

In [12]:
commitment_weight = 1124

# data from open_channel message
feerate_per_kw = 15000
funding_satoshis = 5000000

commitment_fee_rate = int(commitment_weight * feerate_per_kw / 1000)

# TODO double check if we always subtract two times the anchor value or just one time when there is just one anchor output
alice_output_value = funding_satoshis - anchor_output_value - commitment_fee_rate

to_alice_output_value = alice_output_value.to_bytes(8, byteorder="little", signed=True)

outputs = (
    alice_anchor_output_value
    + varint_len(to_alice_anchor_output_spk)
    + to_alice_anchor_output_spk
    + to_alice_output_value
    + varint_len(to_alice_output_spk)
    + to_alice_output_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_tx: ", unsigned_tx.hex())



b' \xa8\xcf\x93'
unsigned_tx:  0200000001f5500ec02b6a685e1f3540f23ec8c47b89ded0084cb064101c2b722fcb28ee6a010000000080df65d3024a01000000000000220020b3df32d74b2af549738f79129baa882f081dbf6d2807a2ee11f0f8bdc72366261a084c0000000000220020b3df32d74b2af549738f79129baa882f081dbf6d2807a2ee11f0f8bdc723662620a8cf93
